In [5]:
!pip install -q catboost

You should consider upgrading via the '/Users/kento/.pyenv/versions/3.8.11/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import catboost as cb
import catboost.datasets as cbd
import numpy as np
import pandas as pd

# print module versions for reproducibility
print('CatBoost version {}'.format(cb.__version__))
print('NumPy version {}'.format(np.__version__))
print('Pandas version {}'.format(pd.__version__))

CatBoost version 1.0.6
NumPy version 1.21.5
Pandas version 1.4.1


In [3]:
print(cbd.adult.__doc__)


    Download "Adult Data Set" [1] from UCI Machine Learning Repository.

    Will return two pandas.DataFrame-s, first with train part (adult.data) and second with test part
    (adult.test) of the dataset.

    [1]: https://archive.ics.uci.edu/ml/datasets/Adult
    


In [4]:
def get_fixed_adult():
    train, test = cbd.adult()
    
    # CatBoost doesn't support pandas.DataFrame missing values for categorical features out 
    # of the box (seed issue #571 on GitHub or issue MLTOOLS-2785 in internal tracker). So 
    # we have to replace them with some designated string manually. 
    for dataset in (train, test, ):
        for name in (name for name, dtype in dict(dataset.dtypes).items() if dtype == np.object):
            dataset[name].fillna('nan', inplace=True)
    
    X_train, y_train = train.drop('income', axis=1), train.income
    X_test, y_test = test.drop('income', axis=1), test.income
    return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, _, _ = get_fixed_adult()

/var/folders/vm/2cthdz497bs3hgjtwrv2qmpm0000gn/T/ipykernel_54142/2445291177.py:8: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  for name in (name for name, dtype in dict(dataset.dtypes).items() if dtype == np.object):


In [6]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba


In [7]:
model = cb.CatBoostClassifier(
    class_names=('<=50K', '>50K'),
    loss_function='Logloss',
    eval_metric='AUC', 
    custom_metric=['AUC'],
    iterations=100,
    random_seed=20181224,
    learning_rate=0.4234185321620083, 
    depth=5, 
    l2_leaf_reg=9.464266235679002)
model.fit(
    cb.Pool(X_train, y_train, cat_features=np.where(X_train.dtypes != np.float)[0]),
    verbose=False)

/var/folders/vm/2cthdz497bs3hgjtwrv2qmpm0000gn/T/ipykernel_54142/937551782.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cb.Pool(X_train, y_train, cat_features=np.where(X_train.dtypes != np.float)[0]),


In [8]:
model.save_model('adult.cbm')

In [9]:
!du -sh adult.cbm

 72K	adult.cbm
